In [1]:
using BinBencherBackend
#using VambBenchmarks
# Load in the data
#sample = "Oral"
#ref = Reference("/home/projects/ku_00197/data/ptracker/tmp/outdir_refs/ref_spades_Oral.json")
#subset!(ref; genomes = is_organism)

In [2]:
## Vamb fra andet run
ref = open(i -> Reference(i), "/home/projects/ku_00197/data/ptracker/tmp/outdir_refs_ef/ref_spades_Airways.json")
subset!(ref;
           genomes = is_organism
       )
final = "/home/projects/ku_00197/data/ptracker/tmp/errorfree/Airways/vambout/vae_clusters.tsv"
#final = "/home/projects/ku_00197/data/ptracker/tmp/errorfree/Airways/vambout/vae_clusters.tsv"
bins = open(i -> Binning(i, ref), final)


 #print_matrix(bins; level=1, assembly=false)
 #print_matrix(bins; level=1, assembly=true)

Binning
  Reference
    Genomes:    1394
    Sequences:  1613696
    Ranks:      9
    Seq length: 56
    Assembled:  32.0 %
  Bins:        12394
  NC genomes:  17
  Precisions: [0.6, 0.7, 0.8, 0.9, 0.95, 0.99]
  Recalls:    [0.6, 0.7, 0.8, 0.9, 0.95, 0.99]
  Recoverable genomes: [427, 350, 267, 157, 106, 0]
  Reconstruction (assemblies):
    P\R   0.6  0.7  0.8  0.9 0.95 0.99
    0.6    69   57   44   31   16    2
    0.7    63   54   41   29   16    2
    0.8    54   45   37   25   15    2
    0.9    39   33   29   19   10    2
    0.95   34   30   27   18    9    2
    0.99   20   19   19   13    6    1


In [25]:
# Vamb fra mit run

ref = open(i -> Reference(i), "/home/projects/ku_00197/data/ptracker/tmp/outdir_refs_ef/ref_spades_Airways.json")
final = "/home/projects/ku_00197/people/lasdan/contig_cutoff/vamb_ptracker_done/test2/formatted_unsplit.tsv"
#bins = Binning(final, ref)
bins = open(i -> Binning(i, ref), final)




Binning
  Reference
    Genomes:    2341
    Sequences:  1613696
    Ranks:      9
    Seq length: 56
    Assembled:  32.4 %
  Bins:        9227
  NC genomes:  66
  Precisions: [0.6, 0.7, 0.8, 0.9, 0.95, 0.99]
  Recalls:    [0.6, 0.7, 0.8, 0.9, 0.95, 0.99]
  Recoverable genomes: [849, 753, 635, 481, 371, 0]
  Reconstruction (assemblies):
    P\R   0.6  0.7  0.8  0.9 0.95 0.99
    0.6   157  133  115   87   71   44
    0.7   147  127  109   83   68   43
    0.8   134  116  103   77   65   42
    0.9   118  106   96   74   62   40
    0.95  106   99   93   73   61   40
    0.99   83   78   76   62   54   39


In [83]:
using DataFrames
using TidierData
function main()
    global df = DataFrame(nc_strains = Any[], vamb_run = Any[], contig_cutoff = Any[], location = Any[])
    for sample in ["Airways", "Oral", "Gastrointestinal", "Skin", "Urogenital"]
        ref = Reference("/home/projects/ku_00197/data/ptracker/tmp/outdir_refs_ef/ref_spades_$sample.json")
        for para in [500,1000, 1500, 2000]
        #for para in [2000]
            for i in 1:5
                vamb_run = string(i)
                #para = "2000"
                path =  "/home/people/lasdan/contig_cutoff/vamb_ptracker_done/$sample/para_$para/"*vamb_run*"_vamb/"
                cluster_path = path * "vae_clusters_split.tsv"
                temppath = path * "formatted_vae_cluster_split.tsv"
                final = path * "formatted_clusters.tsv"
                
                # format vamboutput correctly
                command = `bash /home/people/lasdan/contig_cutoff/vamb_ptracker_done/format_vambout.sh $cluster_path $sample $final`
                run(command)
                
                bins = Binning(final, ref)
                #println(para,": ", bins.recovered_genomes[2][1,1])
                # Recall, precision. 
                println(para,"---",n_recovered(bins, 0.90, 0.95; level=0))
                nc_strains = n_recovered(bins, 0.90, 0.95; level=0) 
                push!(df, [nc_strains, i, para, sample])
            end
        end
    end
    return df
end

main (generic function with 2 methods)

In [84]:
df1 = main()

500---67
500---71
500---61
500---82
500---77
1000---104
1000---103
1000---101
1000---98
1000---101
1500---116
1500---99
1500---116
1500---115
1500---110
2000---104
2000---104
2000---100
2000---104
2000---103
500---58
500---57
500---53
500---52
500---51
1000---52
1000---55
1000---52
1000---56
1000---57
1500---53
1500---61
1500---52
1500---52
1500---58
2000---52
2000---48
2000---47
2000---47
2000---45
500---109
500---122
500---124
500---118
500---114
1000---112
1000---89
1000---93
1000---95
1000---90
1500---93
1500---93
1500---90
1500---92
1500---89
2000---89
2000---84
2000---84
2000---87
2000---88
500---46
500---40
500---36
500---39
500---43
1000---46
1000---46
1000---47
1000---49
1000---44
1500---50
1500---49
1500---52
1500---55
1500---48
2000---55
2000---58
2000---48
2000---46
2000---54
500---45
500---46
500---43
500---45
500---44
1000---45
1000---49
1000---47
1000---50
1000---41
1500---51
1500---47
1500---49
1500---47
1500---45
2000---50
2000---44
2000---47
2000---43
2000---49


Row,nc_strains,vamb_run,contig_cutoff,location
,Any,Any,Any,Any
1,67,1,500,Airways
2,71,2,500,Airways
3,61,3,500,Airways
4,82,4,500,Airways
5,77,5,500,Airways
6,104,1,1000,Airways
7,103,2,1000,Airways
8,101,3,1000,Airways
9,98,4,1000,Airways


In [80]:

using DataFrames
using TidierData
function main2()
    global df = DataFrame(nc_strains = Any[], vamb_run = Any[], contig_cutoff = Any[], location = Any[])
    for sample in ["Airways", "Oral", "Gastrointestinal", "Skin", "Urogenital"]
        ref = Reference("/home/projects/ku_00197/data/ptracker/tmp/outdir_refs_ef/ref_spades_$sample.json")
        for para in [500,1000, 1500, 2000]
        #for para in [2000]
            for i in 1:5
                vamb_run = string(i)
                path =  "/home/people/lasdan/contig_cutoff/error_free_vamb_cutoff/$sample/cutoff_$para/"*vamb_run*"_vamb/"
                #path =  "/home/people/lasdan/contig_cutoff/vamb_ptracker_done/$sample/para_$para/"*vamb_run*"_vamb/"
                cluster_path = path * "vae_clusters_split.tsv"
                temppath = path * "formatted_vae_cluster_split.tsv"
                final = path * "formatted_clusters.tsv"
                
                # format vamboutput correctly
                command = `bash /home/people/lasdan/contig_cutoff/vamb_ptracker_done/format_vambout.sh $cluster_path $sample $final`
                run(command)
                
                bins = Binning(final, ref)
                #println(para,": ", bins.recovered_genomes[2][1,1])
                # Recall, precision. 
                println(para,"---",n_recovered(bins, 0.90, 0.95; level=0))
                nc_strains = n_recovered(bins, 0.90, 0.95; level=0) 
                push!(df, [nc_strains, i, para, sample])
            end
        end
    end
    return df
end
df2 = main2()

500---58
500---75
500---75
500---78
500---73
1000---93
1000---107
1000---85
1000---96
1000---97
1500---103
1500---107
1500---110
1500---113
1500---101
2000---112
2000---108
2000---105
2000---103
2000---110
500---55
500---55
500---59
500---58
500---56
1000---53
1000---66
1000---60
1000---51
1000---53
1500---51
1500---50
1500---54
1500---47
1500---55
2000---47
2000---48
2000---47
2000---55
2000---53
500---105
500---122
500---120
500---110
500---112
1000---89
1000---91
1000---96
1000---124
1000---92
1500---96
1500---91
1500---91
1500---91
1500---97
2000---90
2000---89
2000---90
2000---89
2000---86
500---38
500---44
500---38
500---38
500---39
1000---49
1000---51
1000---48
1000---49
1000---51
1500---46
1500---51
1500---62
1500---52
1500---49
2000---55
2000---56
2000---57
2000---55
2000---51
500---40
500---42
500---43
500---37
500---41
1000---43
1000---48
1000---49
1000---50
1000---49
1500---49
1500---51
1500---48
1500---47
1500---54
2000---52
2000---45
2000---50
2000---48
2000---46


Row,nc_strains,vamb_run,contig_cutoff,location
,Any,Any,Any,Any
1,58,1,500,Airways
2,75,2,500,Airways
3,75,3,500,Airways
4,78,4,500,Airways
5,73,5,500,Airways
6,93,1,1000,Airways
7,107,2,1000,Airways
8,85,3,1000,Airways
9,96,4,1000,Airways


In [85]:
using CSV
CSV.write("nc_strains_results1", df)
CSV.write("nc_strains_results2", df2)

"nc_strains_results2"

Row,nc_strains,vamb_run,contig_cutoff,location
,Any,Any,Any,Any
1,58,1,500,Airways
2,75,2,500,Airways
3,75,3,500,Airways
4,78,4,500,Airways
5,73,5,500,Airways
6,93,1,1000,Airways
7,107,2,1000,Airways
8,85,3,1000,Airways
9,96,4,1000,Airways


In [34]:
#for location in ["Airways", "Oral", "Gastrointestinal", "Skin"]
for location in ["Urogenital"]
    println(location)
    df = main(location)
end
df

Urogenital
500---45
500---46
500---43
500---45
500---44
1000---45
1000---49
1000---47
1000---50
1000---41
1500---51


Traceback (most recent call last):
  File "/home/projects/ku_00197/people/lasdan/contig_cutoff/vamb_ptracker_done/add_sample_vambout.py", line 30, in <module>
    id = ref.split('C')[1]
         ^^^^^^^^^^^^^^
KeyboardInterrupt


LoadError: InterruptException:

Binning
  Reference
    Genomes:    2341
    Sequences:  2605846
    Ranks:      9
    Seq length: 56
    Assembled:  29.2 %
  Bins:        52718
  NC genomes:  7
  Precisions: [0.6, 0.7, 0.8, 0.9, 0.95, 0.99]
  Recalls:    [0.6, 0.7, 0.8, 0.9, 0.95, 0.99]
  Recoverable genomes: [739, 625, 512, 360, 250, 0]
  Reconstruction (assemblies):
    P\R   0.6  0.7  0.8  0.9 0.95 0.99
    0.6    21   15   11   10    8    8
    0.7    19   14   11   10    8    8
    0.8    19   14   11   10    8    8
    0.9    19   14   11   10    8    8
    0.95   19   14   11   10    8    8
    0.99   19   14   11   10    8    8


In [271]:
ref = Reference("/home/projects/ku_00197/data/ptracker/tmp/outdir_refs/ref_spades_Oral.json")

path =  "/home/people/lasdan/contig_cutoff/error_free_vamb_cutoff/Airways/cutoff_2000/1_vamb/"
cluster_path = path * "vae_clusters_unsplit.tsv"
temppath = path * "formatted_vae_cluster_split.tsv"
final = path * "formatted_clusters.tsv"

# format vamboutput correctly
command = `bash /home/people/lasdan/contig_cutoff/vamb_ptracker_done/format_vambout.sh $cluster_path $temppath Oral $final`
run(command)

bins = Binning(final, ref)
println(para,": ", bins.recovered_genomes[2][1,1])

2000: 3


In [269]:
bins

Binning
  Reference
    Genomes:    1324
    Sequences:  2322538
    Ranks:      9
    Seq length: 56
    Assembled:  35.6 %
  Bins:        10802
  NC genomes:  3
  Precisions: [0.6, 0.7, 0.8, 0.9, 0.95, 0.99]
  Recalls:    [0.6, 0.7, 0.8, 0.9, 0.95, 0.99]
  Recoverable genomes: [455, 398, 329, 217, 142, 0]
  Reconstruction (assemblies):
    P\R   0.6  0.7  0.8  0.9 0.95 0.99
    0.6     6    5    5    5    5    3
    0.7     6    5    5    5    5    3
    0.8     6    5    5    5    5    3
    0.9     6    5    5    5    5    3
    0.95    6    5    5    5    5    3
    0.99    6    5    5    5    5    3


In [175]:
n_recovered(bins, 0.66, 0.95; level=1)

17

In [545]:
ref = Reference("/home/projects/ku_00197/data/ptracker/tmp/outdir_refs/ref_spades_Oral.json")
#subset!(ref;
#           genomes = is_plasmid
#       )
#final = "/home/people/lasdan/contig_cutoff/vamb_ptracker_done/test/formatted_clusters.tsv"
#bins = Binning(final, ref)
#println(bins)


Reference
  Genomes:    1324
  Sequences:  2322538
  Ranks:      9
  Seq length: 56
  Assembled:  35.6 %

In [119]:
list::Array = []
for (key,val) in ref.targets_by_name
    if ! isempty(val[2])

        global a = val
        append!(list, [key])
    end
end
set = Set(list)

Set{Any} with 1742212 elements:
  "S7C63427"
  "S4C376221"
  "S11C86973"
  "S8C61334"
  "S27C274108"
  "S9C60387"
  "S10C168953"
  "S11C77004"
  "S7C318076"
  "S8C54557"
  "S11C9222"
  "S7C83681"
  "S4C170962"
  "S7C30308"
  "S10C176140"
  "S7C151061"
  "S27C359617"
  "S7C73737"
  "S9C190496"
  "S4C381762"
  "S11C102906"
  "S11C1560"
  "S8C78764"
  "S4C200219"
  "S4C203076"
  ⋮ 

In [156]:
"S23C40" ∈ keys(ref.targets_by_name)
ref.targe
ts_by_name["S4C1"]

S4C572168 GGCTCCGT
S4C298348 

(Sequence("S4C1", 301156), Tuple{Source{Genome}, UnitRange{Int64}}[(Source(CP004047.1, 3392564), 2764858:3066013)])

In [526]:
using DataFrames
using TidierData
df = DataFrame(bin = Any[], genome = Any[], contig_len = Any[], from_contig = Any[])

Row,bin,genome,contig_len,from_contig
,Any,Any,Any,Any


In [543]:
ref.targets_by_name["S13C116"][2]#[1][1].assembly_size
#ref.targets_by_name["S4C531"][2][1][1]   #[1]#[1][1]

LoadError: KeyError: key "S13C116" not found

In [26]:
ref = Reference("/home/projects/ku_00197/data/ptracker/tmp/outdir_refs/ref_spades_Airways.json")
final = "/home/projects/ku_00197/data/ptracker/tmp/errorfree/Airways/vambout/vae_clusters.tsv"

"/home/projects/ku_00197/data/ptracker/tmp/errorfree/Airways/vambout/vae_clusters.tsv"

In [87]:
using DataFrames
using TidierData

df = DataFrame(bin = Any[], genome = Any[], contig_len = Any[], from_contig = Any[], typ = Any[], genome_size = Any[], assembly_size = Any[])

ref = Reference("/home/projects/ku_00197/data/ptracker/tmp/outdir_refs/ref_spades_Airways.json")
final = "/home/projects/ku_00197/data/ptracker/tmp/errorfree/Airways/vambout/vae_clusters.tsv"
#path =  "/home/people/lasdan/contig_cutoff/error_free_vamb_cutoff/Airways/cutoff_2000/1_vamb/"
#final = path * "formatted_clusters.tsv"
#final = "/home/projects/ku_00197/people/lasdan/contig_cutoff/vamb_ptracker_done/test2/formatted_unsplit.tsv"

dict = Dict()
global i = 0
f = open(final, "r")
for lines in readlines(f)
    id = split(lines)[2]
    bin = split(lines)[1]

    contig_len = ref.targets_by_name[id][1].length 

    
    global tst = ref.targets_by_name[id][2]


    for sos in ref.targets_by_name[id][2]
        flag = sos[1].genome.flags
        #println(sos[1].genome)
        genome = sos[1].genome
        genome_size = genome.genome_size
        assembly_size = genome.assembly_size
        is_genome = Flags.organism in flag
        #assembly_size = sos[1].assembly_size
        push!(df, [bin, genome, contig_len, id, is_genome, genome_size, assembly_size])
        global i += 1
        #println("| ", sos[1].genome) 
        #println()
        #if bin == "S4C531" && genome == "Genome(OTU_97.34032.1)"
        #    print(lines)
    end
    
    #if strip(id) ∉ keys(ref.targets_by_name)
    #    println(id)
    #end
    
    #print(id," " ,bin," ")
    #if  id  ∈ set
    #    
    #    i += 1
    #    for sos in ref.targets_by_name[id][2]
    #        flag = sos[1].genome.flags
    #        #print(Flags.plasmid in flag)
    #        dict[bin] = sos[1].genome
    #        #println("| ", sos[1].genome)
    #    end
    #else
    #    println("not", id)
    #    i += 1
    #end
    #if i == 20
    #    print("break")
    #    break
    #end
end

LoadError: type Set has no field assembly_size

In [33]:
@chain df begin 
    @filter(bin == "vae_611")
end



Row,bin,genome,contig_len,from_contig,typ,genome_size
,Any,Any,Any,Any,Any,Any
1,vae_611,Genome(st_FN649417.1),1948,S23C2783,false,67569
2,vae_611,Genome(OTU_97.1661.0),1948,S23C2783,true,5217911
3,vae_611,Genome(OTU_97.1661.0),1948,S23C2783,true,5217911
4,vae_611,Genome(st_CP017439.1),1948,S23C2783,false,93835
5,vae_611,Genome(OTU_97.2104.1),1948,S23C2783,true,5484407
6,vae_611,Genome(OTU_97.2104.1),1948,S23C2783,true,5484407
7,vae_611,Genome(OTU_97.2104.1),1948,S23C2783,true,5484407
8,vae_611,Genome(OTU_97.2104.1),1948,S23C2783,true,5484407
9,vae_611,Genome(OTU_97.2104.1),1948,S23C2783,true,5484407


In [5]:
[x[1].genome.name for x in ref.targets_by_name["S23C2783"][2]]

7-element Vector{String}:
 "OTU_97.1661.0"
 "OTU_97.1661.0"
 "OTU_97.2104.1"
 "OTU_97.2104.1"
 "OTU_97.2104.1"
 "OTU_97.2104.1"
 "OTU_97.2104.1"

In [35]:
#print(@filter(df, bin == "vae_1018"))
    for sos in ref.targets_by_name[id][2]
        flag = sos[1].genome.flags
        #println(sos[1].genome)
        genome = sos[1].genome
        is_genome = Flags.organism in flag
        #assembly_size = sos[1].assembly_size
        push!(df, [bin, genome, contig_len, id, is_genome])
        global i += 1
        #println("| ", sos[1].genome) 
        #println()
        #if bin == "S4C531" && genome == "Genome(OTU_97.34032.1)"
        #    print(lines)
        #end

    end

LoadError: UndefVarError: `id` not defined

In [484]:
df = DataFrame(bin = Any[], genome = Any[], contig_len = Any[], from_contig = Any[])
push!(df, ["A", "G1", 2, 1])
push!(df, ["A", "G2", 2, 1])
push!(df, ["A", "G3", 2, 2])
push!(df, ["A", "G3", 2, 1])
push!(df, ["A", "G3", 2, 1])
#push!(df, ["A", "G1", 2, 2])
#push!(df, ["A", "G3", 2, 3])
#push!(df, ["B", "G1", 2, 3])
#push!(df, ["B", "G2", 2, 1])
#push!(df, ["B", "G3", 2, 2])
#push!(df, ["B", "G3", 2, 1])
#push!(df, ["B", "G4", 2, 3])
#push!(df, ["B", "G3", 2, 1])

Row,bin,genome,contig_len,from_contig
,Any,Any,Any,Any
1,A,G1,2,1
2,A,G2,2,1
3,A,G3,2,2
4,A,G3,2,1
5,A,G3,2,1


In [55]:
@chain df begin 
    @filter(bin == "vae_611")
    @group_by(from_contig, genome)
    @summarise( genome = first(genome), 
                contig_len = first(contig_len),
                bin = first(bin),
                from_contig = first(from_contig),
                typ = first(typ),
    )
end

    #6708
#@filter(df3, bin == "S4C531")

Row,from_contig,genome,contig_len,bin,typ
,Any,Any,Int64,SubStrin…,Bool
1,S23C2783,Genome(st_FN649417.1),1948,vae_611,false
2,S23C2783,Genome(OTU_97.1661.0),1948,vae_611,true
3,S23C2783,Genome(st_CP017439.1),1948,vae_611,false
4,S23C2783,Genome(OTU_97.2104.1),1948,vae_611,true


In [9]:

@chain df_with_contig_len begin 
    @filter(bin == "vae_65")
end
#@chain df_unique begin 
#    @filter(bin == "vae_65")
#end
df

Row,bin,genome,contig_len,from_contig,typ,genome_size
,Any,Any,Any,Any,Any,Any
1,vae_1,Genome(OTU_97.588.0),3106,S27C4450,true,3785755
2,vae_2,Genome(OTU_97.21926.0),2361,S12C1544,true,2772261
3,vae_3,Genome(OTU_97.39888.0),2701,S7C2371,true,1809844
4,vae_3,Genome(OTU_97.44564.0),2701,S7C2371,true,1814118
5,vae_3,Genome(OTU_97.23667.1),2701,S7C2371,true,1923780
6,vae_4,Genome(OTU_97.22413.1),2971,S12C1209,true,2494543
7,vae_5,Genome(OTU_97.35048.0),3199,S27C4277,true,2530587
8,vae_6,Genome(OTU_97.22413.1),3110,S12C1157,true,2494543
9,vae_7,Genome(OTU_97.24447.1),2396,S9C3566,true,2822480


In [80]:

# Removes copies of the same genomes for a each contigs
df_unique = @chain df begin 
    @group_by(from_contig, genome)
    @summarise( genome = first(genome), 
                contig_len = first(contig_len),
                bin = first(bin),
                from_contig = first(from_contig),
                typ = first(typ),
                genome_size = first(genome_size),
                assembly_size = first(assembly_size)
    )
    @ungroup()
end

# Get the total contig length for each bin and add it to the normal dataframe
df_with_contig_len = @chain df_unique begin
    @group_by(bin, from_contig)
    @summarise(contig_len = first(contig_len)
        )
    @ungroup()
    
    @group_by(bin)
    @summarise(
        total_bin_contig_len = sum(contig_len)
        )
    @ungroup()
    @right_join(df_unique, bin)
end

Row,bin,total_bin_contig_len,from_contig,genome,contig_len,typ,genome_size,assembly_size
,SubStrin…,Int64?,Any,Any,Int64,Bool,Int64,Int64
1,vae_1,3106,S27C4450,Genome(OTU_97.588.0),3106,true,3785755,3710335
2,vae_2,2361,S12C1544,Genome(OTU_97.21926.0),2361,true,2772261,2702457
3,vae_3,2701,S7C2371,Genome(OTU_97.39888.0),2701,true,1809844,1625377
4,vae_3,2701,S7C2371,Genome(OTU_97.44564.0),2701,true,1814118,1748556
5,vae_3,2701,S7C2371,Genome(OTU_97.23667.1),2701,true,1923780,1311379
6,vae_4,2971,S12C1209,Genome(OTU_97.22413.1),2971,true,2494543,2409619
7,vae_5,3199,S27C4277,Genome(OTU_97.35048.0),3199,true,2530587,2447239
8,vae_6,3110,S12C1157,Genome(OTU_97.22413.1),3110,true,2494543,2409619
9,vae_7,2396,S9C3566,Genome(OTU_97.24447.1),2396,true,2822480,2780509


In [2]:


# Get the total contig length for each genome in each bin
df2 = @chain df_with_contig_len begin
    @group_by(bin, genome)
    @summarise(genome_contig_len = sum(contig_len),
               total_bin_contig_len = first(total_bin_contig_len),
        genome_size = first(genome_size),
        assembly_size = first(assembly_size)
        ) # Save the total bin length af the summarise
    @ungroup()
end

# Calculate the percent size of each contigs for each genome w.r.t the total contig size of the bin
df3 = @chain df2 begin
    @group_by(bin)
    @mutate(
        precision = genome_contig_len/total_bin_contig_len,
        #recall = genome_contig_len/assembly_size
        recall = genome_contig_len/genome_size
        #FP = (total_bin_contig_len-genome_contig_len),
        #TP = (assembly_size/genome_size),
    )
    #@mutate(FN = genome_size - TP)
    #@mutate(
        #TPR = TP/(TP+FN),
    #)
    @ungroup()
    @arrange(desc(precision))
    #@filter(percent > 0.95)
end

# and the percent size of each
df4 = @chain df3 begin
    @group_by(bin)
    @filter(precision ==  maximum(precision))
    @ungroup()
    @arrange(desc(precision),desc(recall))
    #@filter(precision > 0.95, recall > 0.9 )
    #@filter( recall > 0.9 )
    @filter(precision > 0.95)
end
#93182/130000
#df4
#23062/28113


LoadError: LoadError: UndefVarError: `@chain` not defined
in expression starting at In[2]:4

In [76]:
df3 = @chain df2 begin
    @group_by(bin)
    @mutate(percent = genome_contig_len/genome_size)
    @ungroup()
    @arrange(desc(percent))
    #@filter(percent > 0.95)
end

# and the percent size of each
df4 = @chain df3 begin
    @group_by(bin)
    @filter(percent ==  maximum(percent))
    @ungroup()
    @arrange(desc(percent))
    #@filter(percent > 0.95)
end


Row,bin,genome,genome_contig_len,total_bin_contig_len,genome_size,percent
,SubStrin…,Any,Int64,Int64,Int64,Float64
1,vae_209,Genome(st_AE015932.1),15065,24221958,8198,1.83764
2,vae_251,Genome(OTU_97.34663.1),3514421,25337472,2691660,1.30567
3,vae_112,Genome(st_AE015931.1),6284,8495575,4828,1.30157
4,vae_159,Genome(OTU_97.45281.1),4045180,16978673,3392564,1.19237
5,vae_99,Genome(OTU_97.17807.0),2706016,18627884,2433571,1.11195
6,vae_62,Genome(st_CP003925.1),50569,4621091,46627,1.08454
7,vae_385,Genome(st_CP007157.1),43331,2215872,42636,1.0163
8,vae_227,Genome(st_CP001728.1),94436,12177343,92982,1.01564
9,vae_187,Genome(st_CP012912.1),11695,8643445,11619,1.00654


LoadError: IOError: open("/home/people/lasdan/.julia/logs/manifest_usage.toml.pid", 194, 292): no space left on device (ENOSPC)

In [392]:
df2 = @chain df begin
    
    @group_by(bin, genome)
    #@summarise(number_genomes_in_bin = n())
    #@mutate(sum_bin = sum(number_genomes_in_bin))
    #@ungroup()

    
    #@group_by(bin)
    #@mutate(percent = number_genomes_in_bin/sum_bin)
    #@filter(percent ==  maximum(percent))
    #@ungroup()
    #@arrange(desc(percent))
    #@filter(sum_bin > 10)
end

Row,bin,genome
,Any,Any
1,A,G2
2,A,G2
Row,bin,genome
,Any,Any
1,B,G3
2,B,G3
3,B,G3


In [141]:
b=(a[2][1])

(Sequence("S7C63427", 437), Tuple{Source{Genome}, UnitRange{Int64}}[(Source(LT635457.1, 1916314), 1367344:1367780)])

In [134]:
b=(a[2][1])
b[1].genome

Genome "st_CP001002.1"
  Parent:        "sp_CP001002.1"
  Genome size:   593579
  Assembly size: 551283 (92.9 %)
  Sources:       1
  Flags:         4 (plasmid)

In [135]:
a

(Sequence("S11C77004", 442), Tuple{Source{Genome}, UnitRange{Int64}}[(Source(CP001002.1, 593579), 79106:79547)])

In [125]:
ref.targets_by_name["S23C40"]

(Sequence("S23C40", 93654), Tuple{Source{Genome}, UnitRange{Int64}}[])

In [107]:
ref.targets_by_name["S7C63427"][2]
ref


Reference
  Genomes:    0
  Sequences:  2605846
  Ranks:      0
  Seq length: 56
  Assembled:  NaN %

In [101]:
?ref

search: ref Ref Reference WeakRef prevfloat chopprefix UndefRefError GlobalRef



No documentation found.

`ref` is of type `Reference`.

# Summary

```
mutable struct Reference
```

# Fields

```
genomes            :: Set{Genome}
targets_by_name    :: Dict{String, Tuple{Sequence, Vector{Tuple{Source{Genome}, UnitRange{Int64}}}}}
clades             :: Vector{Vector{Clade{Genome}}}
shortest_seq_len   :: Union{LazilyInitializedFields.Uninitialized, Int64}
fraction_assembled :: Union{LazilyInitializedFields.Uninitialized, Float64}
```
